In [1]:
import pandas as pd
import pymzml
import numpy as np
import os
import pickle
import pandas as pd
from tqdm import tqdm
import pyteomics
from pyteomics import mzml, auxiliary, mgf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import argparse
import json
from rdkit import Chem

In [2]:
def _read_mgf(mgf_path: str) -> pd.DataFrame:
    msms_df = []
    with mgf.MGF(mgf_path) as reader:
        for spectrum in reader:
            d = spectrum['params']
            d['spectrum'] = np.array([spectrum['m/z array'],
                                      spectrum['intensity array']])
            if 'precursor_mz' not in d:
                d['precursor_mz'] = d['pepmass'][0]
            else:
                d['precursor_mz'] = float(d['precursor_mz'])
            msms_df.append(d)
    msms_df = pd.DataFrame(msms_df)
    return msms_df

In [3]:
result = pd.read_csv('bc1ee579afd14fac8c2651373e00d25b-merged_results_with_gnps.tsv', sep='\t')
result


,SpectrumID,Compound_Name,Ion_Source,Instrument,Compound_Source,PI,Data_Collector,Adduct,Precursor_MZ,ExactMass,...,tags,molecular_formula,InChIKey,InChIKey-Planar,superclass,class,subclass,npclassifier_superclass,npclassifier_class,npclassifier_pathway
0,CCMSLIB00000579274,Stenothricin Analog -42 C46H79N11O17S,LC-ESI,qTof,Lysate,Mingxun Wang,NaN,M+H,1090.540,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCMSLIB00005436027,Stenothricin G,LC-ESI,qTof,Crude,Dorrestein,Nothias,M+H,1160.620,0.000,...,NaN,C51H89N11O17S,DXAUKYYEJIPVHN-BHWJIPFMSA-N,DXAUKYYEJIPVHN,Organic acids and derivatives,Peptidomimetics,Depsipeptides,Oligopeptides,Cyclic peptides|Depsipeptides|Lipopeptides,Amino acids and Peptides|Polyketides
2,CCMSLIB00000075071,Stenothricin D,LC-ESI,LTQ-FTICR,Isolated,Dorrestein,Peng,M+H,1132.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CCMSLIB00000075072,Stenothricin B,LC-ESI,LTQ-FTICR,Isolated,Dorrestein,Peng,M+H,1118.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CCMSLIB00011431457,demo comppound,LC-ESI,qTof,Lysate,mka,,M+H,400.094,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,CCMSLIB00000072151,Phosphatidylethanolamine (22:0/16:1) Abbr: BPoPE,DI-ESI,qTof,Crude,Dorrestein,J Watrous,M+Na,796.458,773.594,...,NaN,C43H84NO8P,ZNMZYHOEYPCVMP-MITQYPHISA-N,ZNMZYHOEYPCVMP,Lipids and lipid-like molecules,Glycerophospholipids,Glycerophosphoethanolamines,Glycerophospholipids,Glycerophosphoethanolamines,Fatty acids
821,CCMSLIB00000579251,trihydroxyflavone C-hexoside,DI-ESI,Orbitrap,Other,RicardoMBorges,RicardoMBorges,M-H,431.098,432.106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,CCMSLIB00000075009,Mureidomycin B,LC-ESI,qTof,Isolated,Dorrestein,Peng,M+H,843.334,842.327,...,NaN,C38H50N8O12S,DXBJHRPKFQGVGZ-UHFFFAOYSA-N,DXBJHRPKFQGVGZ,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Oligopeptides,Linear peptides,Amino acids and Peptides
823,CCMSLIB00010314557,3-O-detigloyl-3-O-benzoyl-12-acetoxyswietephra...,LC-ESI,qTof,Isolated,Milton Silva,Wender Gomes,[M+H],749.281,748.273,...,NaN,C40H44O14,KVZDZCHRKIXZEW-URMGJMHJSA-N,KVZDZCHRKIXZEW,NaN,NaN,NaN,Triterpenoids,Limonoids,Terpenoids


In [6]:
file_path = '32_Mix_Cysteine_OnlineReactivity.mgf'
input = _read_mgf(file_path)
contains_educt = input[~input['online_reactivity'].isna()]

In [45]:
# add column reactivity to result
result['reactivity'] = 0

for index, row in contains_educt.iterrows():
    reaction_smarts = []
    sample = json.loads(row['online_reactivity'])
    for item in sample:
        reaction_smarts.append(Chem.MolFromSmarts(item['educt_smarts']))

    scanNumber = int(row['scans'])
    result_row = result[(result['#Scan#'] == scanNumber) & (~result['Smiles'].isna())]

    for i, match_row in result_row.iterrows():
        try:
            Mol = Chem.MolFromSmiles(match_row['Smiles'])
            for substruct in reaction_smarts:
                if Mol.HasSubstructMatch(substruct):
                    result.loc[i, 'reactivity'] = result.loc[i, 'reactivity'] + 1
        except:
            pass

    

[18:50:11] SMILES Parse Error: syntax error while parsing: 
[18:50:11] SMILES Parse Error: Failed parsing SMILES ' ' for input: ' '


In [47]:
result[result['reactivity'] > 0]

,SpectrumID,Compound_Name,Ion_Source,Instrument,Compound_Source,PI,Data_Collector,Adduct,Precursor_MZ,ExactMass,...,molecular_formula,InChIKey,InChIKey-Planar,superclass,class,subclass,npclassifier_superclass,npclassifier_class,npclassifier_pathway,reactivity


In [48]:
result

,SpectrumID,Compound_Name,Ion_Source,Instrument,Compound_Source,PI,Data_Collector,Adduct,Precursor_MZ,ExactMass,...,molecular_formula,InChIKey,InChIKey-Planar,superclass,class,subclass,npclassifier_superclass,npclassifier_class,npclassifier_pathway,reactivity
0,CCMSLIB00000579274,Stenothricin Analog -42 C46H79N11O17S,LC-ESI,qTof,Lysate,Mingxun Wang,NaN,M+H,1090.540,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,CCMSLIB00005436027,Stenothricin G,LC-ESI,qTof,Crude,Dorrestein,Nothias,M+H,1160.620,0.000,...,C51H89N11O17S,DXAUKYYEJIPVHN-BHWJIPFMSA-N,DXAUKYYEJIPVHN,Organic acids and derivatives,Peptidomimetics,Depsipeptides,Oligopeptides,Cyclic peptides|Depsipeptides|Lipopeptides,Amino acids and Peptides|Polyketides,0
2,CCMSLIB00000075071,Stenothricin D,LC-ESI,LTQ-FTICR,Isolated,Dorrestein,Peng,M+H,1132.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,CCMSLIB00000075072,Stenothricin B,LC-ESI,LTQ-FTICR,Isolated,Dorrestein,Peng,M+H,1118.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,CCMSLIB00011431457,demo comppound,LC-ESI,qTof,Lysate,mka,,M+H,400.094,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,CCMSLIB00000072151,Phosphatidylethanolamine (22:0/16:1) Abbr: BPoPE,DI-ESI,qTof,Crude,Dorrestein,J Watrous,M+Na,796.458,773.594,...,C43H84NO8P,ZNMZYHOEYPCVMP-MITQYPHISA-N,ZNMZYHOEYPCVMP,Lipids and lipid-like molecules,Glycerophospholipids,Glycerophosphoethanolamines,Glycerophospholipids,Glycerophosphoethanolamines,Fatty acids,0
821,CCMSLIB00000579251,trihydroxyflavone C-hexoside,DI-ESI,Orbitrap,Other,RicardoMBorges,RicardoMBorges,M-H,431.098,432.106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
822,CCMSLIB00000075009,Mureidomycin B,LC-ESI,qTof,Isolated,Dorrestein,Peng,M+H,843.334,842.327,...,C38H50N8O12S,DXBJHRPKFQGVGZ-UHFFFAOYSA-N,DXBJHRPKFQGVGZ,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Oligopeptides,Linear peptides,Amino acids and Peptides,0
823,CCMSLIB00010314557,3-O-detigloyl-3-O-benzoyl-12-acetoxyswietephra...,LC-ESI,qTof,Isolated,Milton Silva,Wender Gomes,[M+H],749.281,748.273,...,C40H44O14,KVZDZCHRKIXZEW-URMGJMHJSA-N,KVZDZCHRKIXZEW,NaN,NaN,NaN,Triterpenoids,Limonoids,Terpenoids,0


In [25]:
row = contains_educt.iloc[0]
reaction_smarts = []
if type(row['online_reactivity']) == str:
    sample = json.loads(row['online_reactivity'])
else:
    sample = row['online_reactivity']
    
for item in sample:
    reaction_smarts.append(item['educt_smarts'])

scanNumber = int(row['scans'])
result_row = result[result['scan'] == scanNumber]

In [24]:
row['online_reactivity'].iloc[0]

'[{"compound_type":"Product","reaction_name":"L-cysteine: [M+Na]+ → [M+H]+","filename_contains":"cysteine","educt_smarts":"O=C-1-C([C,H,O])=C([C,H,O])-C(=[O])-c:2:c([*]):c([*]):c([*]):c([*]):c-1:2","reaction_smarts":"O=C-1-C([C,H,O])=C([C,H,O])-C(=[O])-c:2:c([*]):c([*]):c([*]):c([*]):c-1:2.N[C@@H](CS)C(O)=O>>O=C1C(C(C(c2c1c([*])c([*])c([*])c2[*])=O)[#1,#6,#8])(SC[C@@H](C(O)=O)N)[#1,#6,#8]","delta_mz":99.037808,"linked_ids":[331]},{"compound_type":"Product","reaction_name":"L-cysteine: [M+Na]+ → [M+H]+","filename_contains":"cysteine","educt_smarts":"O=C(O[C,H])C([C,H,O])=C([C,H,O])[C,H,O]","reaction_smarts":"O=C(O[C,H])C([C,H,O])=C([C,H,O])[C,H,O].N[C@@H](CS)C(O)=O>>O=C(C(C([#1,#6,#8])(SC[C@@H](C(O)=O)N)[#1,#6,#8])[#1,#6,#8])O[#1,#6]","delta_mz":99.037808,"linked_ids":[331]},{"compound_type":"Product","reaction_name":"L-cysteine: [M+Na]+ → [M+H]+","filename_contains":"cysteine","educt_smarts":"[C]C(C([C,H,O])=C([C,H,O])[C,H,O])=O","reaction_smarts":"[C]C(C([C,H,O])=C([C,H,O])[C,H,O])=O.

In [36]:
scanNumber = int(row['scans'])

In [34]:
scanNumber = 3251


In [37]:
result[result['#Scan#'] == scanNumber]

,SpectrumID,Compound_Name,Ion_Source,Instrument,Compound_Source,PI,Data_Collector,Adduct,Precursor_MZ,ExactMass,...,tags,molecular_formula,InChIKey,InChIKey-Planar,superclass,class,subclass,npclassifier_superclass,npclassifier_class,npclassifier_pathway


In [39]:
result.columns

Index(['SpectrumID', 'Compound_Name', 'Ion_Source', 'Instrument',
       'Compound_Source', 'PI', 'Data_Collector', 'Adduct', 'Precursor_MZ',
       'ExactMass', 'Charge', 'CAS_Number', 'Pubmed_ID', 'Smiles', 'INCHI',
       'INCHI_AUX', 'Library_Class', 'IonMode', 'UpdateWorkflowName',
       'LibraryQualityString', '#Scan#', 'SpectrumFile', 'LibraryName',
       'MQScore', 'Organism', 'TIC_Query', 'RT_Query', 'MZErrorPPM',
       'SharedPeaks', 'MassDiff', 'LibMZ', 'SpecMZ', 'SpecCharge',
       'FileScanUniqueID', 'NumberHits', 'tags', 'molecular_formula',
       'InChIKey', 'InChIKey-Planar', 'superclass', 'class', 'subclass',
       'npclassifier_superclass', 'npclassifier_class',
       'npclassifier_pathway'],
      dtype='object')